# EXAMPLES OF MORPHOLOGICAL OPENING AND CLOSING

This is a short tutorial to explain step by step the morphological opening and closing applied to point clouds according to the work:

- Balado, J., Van Oosterom, P., Díaz-Vilariño, L., & Meijers, M. (2020). Mathematical morphology directly applied to point cloud data. ISPRS Journal of Photogrammetry and Remote Sensing, 168, 208-220.

It is previously recommended to review the morphological dilation and erosion tutorials.


# Import of libraries

The libraries used in the mathematical morphology are *numpy* and *o3d*.

- https://numpy.org/
- http://www.open3d.org/

In [1]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Definition of dilation and erosion operations

As the morphological operations of opening and closing are combinations of dilation and erosion, both functions are defined below without intermediate outputs, whose input parameters are the point cloud to dilate/erosion, the SE point cloud, and the search distance.


In [2]:
def mm_dilate(input_data,SE,d): 

    dilated_data = input_data[:,0:3]

    for i in range(1,SE.shape[0]):
        
        # Move point i from the SE to the whole input cloud
        translated_SE = input_data[:,0:3] + SE[i,:]

        # Convert to SE traslated to cloud-object
        pcd_tSE = o3d.geometry.PointCloud()
        pcd_tSE.points = o3d.utility.Vector3dVector(translated_SE)
        
        # Convert concatenated point cloud to cloud-object
        pcd_dil = o3d.geometry.PointCloud()
        pcd_dil.points = o3d.utility.Vector3dVector(dilated_data)

        # Calculate distances between clouds
        dist_pcd_tSE_2_pcd_dil = pcd_tSE.compute_point_cloud_distance(pcd_dil)
        dist_pcd_tSE_2_pcd_dil = np.asarray(dist_pcd_tSE_2_pcd_dil) 

        # Checking the existence of nearby points
        idx_add = dist_pcd_tSE_2_pcd_dil > d/2
        
        # Adding new points to the dilated cloud
        dilated_data = np.vstack((dilated_data,translated_SE[idx_add,0:3]))
        
    return dilated_data

In [3]:
def mm_erode(input_data,SE,d):
    
    # Convert the input points to point cloud-object
    pcd_in = o3d.geometry.PointCloud()
    pcd_in.points = o3d.utility.Vector3dVector(input_data[:,0:3])
    
    # Generate indices of points to keep
    idx_remain = np.ones(input_data.shape[0], dtype=bool)

    for i in range(1,SE.shape[0]):
        
        # Move point i from the SE to the whole cloud
        translated_SE = input_data[:,0:3] + SE[i,:]

        # Convert to SE moved to point cloud-object
        pcd_tSE = o3d.geometry.PointCloud()
        pcd_tSE.points = o3d.utility.Vector3dVector(translated_SE)

        # Calculate distances between clouds
        dist_pcd_tSE_2_pcd_in = pcd_tSE.compute_point_cloud_distance(pcd_in)
        dist_pcd_tSE_2_pcd_in = np.asarray(dist_pcd_tSE_2_pcd_in)    

        # Filtering by distances of points in the input cloud that match points in the SE
        idx_aux = dist_pcd_tSE_2_pcd_in < d
        
        # Combination with the index list of input data
        idx_remain = idx_remain * idx_aux

    # Selection of the output points according to the indexes of the points to be preserved.
    output_data = input_data[idx_remain,0:3]    
    
    return output_data

# Reading data

As in the previous tutorials, the reading of the input point cloud is done with *numpy*. To visualise the input data in 3D, we resort to the *o3d* library, for which it is necessary to previously transform our point cloud to a point cloud object of the library.

* Note: the predefined axes of the visualisation do not correspond to the real axes of the cloud.


In [4]:
# Reading data
input_data = np.loadtxt("Nubes/cubo3d.txt", delimiter=' ')

In [5]:
# Creat point cloud object
pcd_in = o3d.geometry.PointCloud()

# Load points into the object, delimited to XYZ in case the cloud has more attributes
pcd_in.points = o3d.utility.Vector3dVector(input_data[:,0:3])

# Visualization
o3d.visualization.draw_geometries([pcd_in])

# Define Structuring Element (SE)

In this case we are going to define a more complex structuring element. Although opening and closing are independent operations, we are going to use the same SE for both. Since the morphological opening operations are used to segment and the morphological closing is used to fill gaps, we are going to define an SE with a geometry to:

- Segment the "small" vertical elements of the cube.
- Close the small gap on the front face of the cube.

<center> <img src="Figures/F04.jpg"></center>
<center> Figura 1. Medidas de la nube de entrada (f) </center>

Therefore, since the vertical elements of the cube and the small gap are oriented in the direction of the X-axis, this direction is used for the SE. Also, to segment the vertical elements and eliminate the small area, the SE must be larger than the small area (1 m). To fill the gap, the SE must be larger than the gap (also 1m). The density of the SE is adjusted to have the same density as the input cloud (0.1 m between points), although with practice, the density can be reduced and save execution time. Finally, the SE is centred at [0,0,0], leaving 5 points in the positive direction of X and 5 points in the negative direction, all spaced 0.1 m apart.

In [6]:
# SE definition
SE =  np.array([[0, 0, 0],
      [-0.1, 0, 0],
      [-0.2, 0, 0],
      [-0.3, 0, 0],
      [-0.4, 0, 0],
      [-0.5, 0, 0],
      [0.1, 0, 0],
      [0.2, 0, 0],
      [0.3, 0, 0],
      [0.4, 0, 0],
      [0.5, 0, 0]])

# Define search distance

Since the point density of the SE and the input cloud are equal, we will set the search radius accordingly: 

In [7]:
# Define search radius
d = 0.1

# Morphological opening

The morphological opening is an erosion followed by a dilation, with the same SE and radius of search, where the ouput of the erosion is the input of the dilation.

In [8]:
# Morphological opening
eroded_data = mm_erode(input_data,SE,d)  
opened_data = mm_dilate(eroded_data,SE,d)    

In [9]:
# Visualization
pcd_open = o3d.geometry.PointCloud()
pcd_open.points = o3d.utility.Vector3dVector(opened_data[:,0:3])
o3d.visualization.draw_geometries([pcd_open])

# Export open point cloud

As in the other tutorials, it is also possible to save the point cloud to disk, specifying a directory and name for the txt. 

In [10]:
# Definition of the path and file name
ruta = "Nubes/cubo_open.txt"

# Save
np.savetxt(ruta,opened_data,delimiter=' ') 

# Morphological closing

Morphological closing is a dilation followed by erosion, with the same SE and search radius, where the output of the dilation is the input of the erosion.

In [11]:
# Morphological closing
dilated_data = mm_dilate(input_data,SE,d)
closed_data = mm_erode(dilated_data,SE,d)     

In [12]:
# Visualization 
pcd_close = o3d.geometry.PointCloud()
pcd_close.points = o3d.utility.Vector3dVector(closed_data[:,0:3])
o3d.visualization.draw_geometries([pcd_close])

# Export closed point cloud

As in the other tutorials, it is also possible to save the point cloud to disk, specifying a directory and name for the txt.

In [16]:
# Definition of the path and file name
ruta = "Nubes/cubo_close.txt"

# Save
np.savetxt(ruta,eroded_data,delimiter=' ') 

The combination of morphological dilation, erosion, opening and closing is a powerful tool for processing point clouds. By combining operations with different SE, the desired parts of the point cloud can be segmented or modified. Furthermore, by knowing the geometrical shapes and orientation, many objects can be automatically detected. I invite interested parties to test this code with their own point clouds.